# Building up to an agent
LLM -> Chain -> Agent

In [1]:
%reload_ext autoreload
%autoreload 2

# Load environment variables
import dotenv
dotenv.load_dotenv("../../.env", override=True)

True

# Agents

There are many types of agent that work in slightly different ways, but the main concepts behind an agent are:

1. The agent has access to tools. Tools include instructions on how to use them and what they are useful for. A tool can be anything that could be used by using text.
2. The agent has a task, usually this is just the prompt from the user.
3. The agent has access to an LLM and a system prompt that tells it how to go about achieving its task given its tools
4. The agent will attempt to break down its task according to its programming and iteratively take steps to achieve it. The agent will try to use the tools at its disposal when it makes sense and (usually) will eventually decide it is done and return a final output.

In [40]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from pandasai import SmartDataframe
import pandas as pd
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.tools import DuckDuckGoSearchRun

from pydantic import BaseModel, Field


llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

# Smart Dataframe Tool
class FaultTolerantSmartDataframe:
    def __init__(self, df, config):
        self.sdf = SmartDataframe(df, config=config)
    
    def __call__(self, prompt):
        try:
            return self.sdf.chat(prompt)
        except Exception as e:
            return f'Error: {e}'
        

df = pd.read_csv('../extra_resources/candy-data.csv')
sdf = FaultTolerantSmartDataframe(df, config={"llm": llm})
    
candy_data_tool = Tool.from_function(
    func=sdf,
    name="Candy Dataset",
    description="Useful to get general information about candies, their classification, price, price and rating. You can ask it questions in natural language and it will answer back.",
    # args_schema=CodeSuggestionsInput
)

# Search tool
search = DuckDuckGoSearchRun()

# Init agent
from langchain.agents import initialize_agent

model = ChatOpenAI(temperature=0)
tools = [candy_data_tool, search]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True)


In [46]:
agent.run('I want to buy a healthy can that is also cheap. Use the candy dataset to select 3 healthy candies with the lowest price. Then search these online and tell me which is the currently cheapest candy of the three. Think step by step.')



> Entering new AgentExecutor chain...
First, I need to use the Candy Dataset to find 3 healthy candies with the lowest price. Then, I can search online to find the currently cheapest candy among the three selected.
Action: Candy Dataset
Action Input: Find 3 healthy candies with the lowest price
Observation:           competitorname  chocolate  fruity  caramel  peanutyalmondy  nougat  \
76  Tootsie Roll Midgies          1       0        0               0       0   
48          Pixie Sticks          0       0        0               0       0   
15           Fruit Chews          0       1        0               0       0   

    crispedricewafer  hard  bar  pluribus  sugarpercent  pricepercent  \
76                 0     0    0         1         0.174         0.011   
48                 0     0    0         1         0.093         0.023   
15                 0     0    0         1         0.127         0.034   

    winpercent  
76   45.736748  
48   37.722336  
15   43.088924  
Thought

'The currently cheapest candy among the three healthy candies with the lowest price is Pixie Sticks.'

# [Toolkits](https://python.langchain.com/docs/modules/agents/toolkits/)
### [Document Comparisons](https://python.langchain.com/docs/integrations/toolkits/document_comparison_toolkit)
- Tool that wraps full documents. This is an easy way to do document QA on a short list of documents.
- Includes PDF reader.
### [Vectorstore](https://python.langchain.com/docs/integrations/toolkits/vectorstore)
- Interface with arbitrary vector store
### [JSON](https://python.langchain.com/docs/integrations/toolkits/json)
- Tool for interacting with large JSON/dict objects.
### [OpenAPI](https://python.langchain.com/docs/integrations/toolkits/openapi)
- Tool to consume arbitrary APIs, here APIs conformant to the OpenAPI/Swagger specification.
### [Natural Language APIs](https://python.langchain.com/docs/integrations/toolkits/openapi_nla)
- efficiently plan and combine calls across endpoints using natural language as an intermediary
### Specific Services
- [Github](https://python.langchain.com/docs/integrations/toolkits/github)
- [Office 365](https://python.langchain.com/docs/integrations/toolkits/office365)
- [ClickUp - (WIP / georgian made.)](https://github.com/langchain-ai/langchain/pull/10662)

In [3]:
import boto3
import json
bedrock = boto3.client(service_name='bedrock-runtime')

body = json.dumps({
    "prompt": "\n\nHuman:explain cats to 8th graders\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.1,
    "top_p": 0.9,
})

modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())
print(response_body.get('completion'))

 Here is how I would explain cats to 8th grade students:

Cats are small, furry mammals that are kept as pets by many people. There are over 70 different breeds of domestic cats. Some common cat breeds include Siamese, Persian, Maine Coon, and tabby cats. 

Cats have several unique features that help them survive as predators:

- Retractable claws - Cats can extend and retract their sharp claws to help them climb, hunt prey, and defend themselves. Their claws are attached to the last bone of each toe.

- Whiskers - The long, stiff hairs on a cat's face are called whiskers. They help cats detect objects and navigate in the dark. 

- Excellent night vision - Cats can see well in dim light thanks to extra reflecting cells in their eyes called the tapetum lucidum. This layer helps them hunt at night.

- Flexible bodies - Cats are extremely agile. They can twist, jump, climb, and squeeze through very small spaces thanks to their flexible spines. This helps them climb trees and hunt prey.

-

# Hot Off the Press 🔥
** WARNING: There will be bugs **

## Experimentation - [Langsmith](https://www.langchain.com/langsmith)

- Register datasets
- Log chain and agent tracing
- Keep track of latency and other metrics